In [8]:
import subprocess
import os

def check_ffmpeg_and_ffprobe():
    try:
        # Check ffmpeg
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
        # Check ffprobe 
        subprocess.run(['ffprobe', '-version'], capture_output=True, check=True)
        return True
    except (subprocess.SubprocessError, FileNotFoundError):
        print("FFmpeg and/or FFprobe are not installed or not in PATH. Please run ./setup.sh from the project root to install them.\n"
              "Alternatively, you can install manually:\n"
              "- On macOS: brew install ffmpeg\n"
              "- On Ubuntu/Debian: sudo apt install ffmpeg\n"
              "- On Windows: Download from https://ffmpeg.org/download.html")
        
        return False

# Run the function and print result
is_ffmpeg_installed = check_ffmpeg_and_ffprobe()
print(f"FFmpeg and FFprobe are installed: {is_ffmpeg_installed}")


FFmpeg and FFprobe are installed: True


In [11]:
import yt_dlp

def download_youtube_audio(url, output_path=None):
    """
    Download audio from a YouTube video or playlist
    
    Args:
        url (str): YouTube video or playlist URL
        output_path (str, optional): Path where the audio should be saved.
            For single videos: filename.mp3
            For playlists: playlist_name/video_title.mp3
    """
    # First check if ffmpeg and ffprobe are installed
    if not check_ffmpeg_and_ffprobe():
        return
        
    # Get ffmpeg location
    try:
        ffmpeg_path = subprocess.check_output(['which', 'ffmpeg']).decode().strip()
        ffmpeg_dir = os.path.dirname(ffmpeg_path)
    except subprocess.SubprocessError:
        print("Could not locate ffmpeg. Please ensure it's installed and in your PATH")
        return

    # Create downloads directory if it doesn't exist
    downloads_dir = os.path.join(os.path.dirname(os.getcwd()), 'downloads')
    os.makedirs(downloads_dir, exist_ok=True)

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'ffmpeg_location': ffmpeg_dir,  # Add ffmpeg directory location
        'quiet': False,
        'no_warnings': False,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Extract info first to determine if it's a playlist
            info = ydl.extract_info(url, download=False)
            
            if 'entries' in info:  # It's a playlist
                # Create playlist directory
                playlist_name = info.get('title', 'playlist')
                playlist_dir = os.path.join(downloads_dir, playlist_name)
                os.makedirs(playlist_dir, exist_ok=True)
                
                # Set output template for playlist items
                ydl_opts['outtmpl'] = os.path.join(playlist_dir, '%(title)s.%(ext)s')
            else:  # Single video
                if output_path:
                    ydl_opts['outtmpl'] = os.path.join(downloads_dir, output_path)
                else:
                    ydl_opts['outtmpl'] = os.path.join(downloads_dir, '%(title)s.%(ext)s')
            
            # Download with updated options
            with yt_dlp.YoutubeDL(ydl_opts) as ydl_download:
                ydl_download.download([url])
                
        print("Audio download completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Example usage

# Can be video url or playlist url
video_url = "https://www.youtube.com/playlist?list=PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS"
download_youtube_audio(video_url)

# If you want to specify an output path (for single videos only):
# download_youtube_audio("https://www.youtube.com/watch?v=example", "my_audio.mp3")

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS
[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS: Downloading webpage
[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Toad Voice Clips
[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] Playlist Toad Voice Clips: Downloading 24 items of 24
[download] Downloading item 1 of 24
[youtube] Extracting URL: https://www.youtube.com/watch?v=wQ3Ijr-Qhz4
[youtube] wQ3Ijr-Qhz4: Downloading webpage
[youtube] wQ3Ijr-Qhz4: Downloading ios player API JSON
[youtube] wQ3Ijr-Qhz4: Downloading mweb player API JSON
[youtube] wQ3Ijr-Qhz4: Downloading m3u8 information
[download] Downloading item 2 of 24
[youtube] Extracting URL: https://www.youtube.com/watch?v=aoKvzcK06AE
[youtube] aoKvzcK06AE: Downloading webpage
[youtube] aoKvzcK06AE: Downloading ios player API JSON
[youtube] aoKvzcK06AE: Downloading mweb player API JSON
[youtube] aoKvzcK06AE: Downloading m3u8 information
[download] Downloading item 3 of 24
[youtube] Extracting URL: https://www.youtube.com/watch?v=Y50lq-1JrSQ
[youtube] Y50lq-1JrSQ: Downloading webpage
[youtube] Y50lq-1JrSQ: Downloading ios player API JSON
[youtube] Y50lq-1JrSQ: Downloading mweb player API JSON
[youtube] Y50lq-1JrSQ: Downloading m3u8 informat

[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] PLT7YQtwNLtQ1NVxBXfSkkoU5V3kZjU2uS page 1: Downloading API JSON


[youtube:tab] Playlist Toad Voice Clips: Downloading 24 items of 24
[download] Downloading item 1 of 24
[youtube] Extracting URL: https://www.youtube.com/watch?v=wQ3Ijr-Qhz4
[youtube] wQ3Ijr-Qhz4: Downloading webpage
[youtube] wQ3Ijr-Qhz4: Downloading ios player API JSON
[youtube] wQ3Ijr-Qhz4: Downloading mweb player API JSON
[youtube] wQ3Ijr-Qhz4: Downloading m3u8 information
[info] wQ3Ijr-Qhz4: Downloading 1 format(s): 251
[download] /Users/suchintan/Development/audiobook-generator/downloads/Toad Voice Clips/Mario Kart 8 - All Toad Sound Effects ⧸ Voice Clips.webm has already been downloaded
[download] 100% of    1.04MiB
[ExtractAudio] Destination: /Users/suchintan/Development/audiobook-generator/downloads/Toad Voice Clips/Mario Kart 8 - All Toad Sound Effects ⧸ Voice Clips.mp3
Deleting original file /Users/suchintan/Development/audiobook-generator/downloads/Toad Voice Clips/Mario Kart 8 - All Toad Sound Effects ⧸ Voice Clips.webm (pass -k to keep)
[download] Downloading item 2 of 24